In [1]:
import tensorflow as tf
import pathlib
import os
# import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly

np.set_printoptions(precision=4)

In [2]:
DATA_LOCATION="/data/kaggle/zindi/fruit_disease_detection"
TRAIN_IMAGES=os.path.join(DATA_LOCATION,"Train_Images")
CSV_LOCATION=os.path.join(DATA_LOCATION,"Train.csv")
YOLO_CSV='train_yolo.csv'

In [3]:
dataset = tf.data.Dataset.list_files(TRAIN_IMAGES+"/*.jpg")

In [4]:
#read YOLO df
df=pd.read_csv(YOLO_CSV)
df.head()

,Unnamed: 0,Image_ID,class,xmin,ymin,width,height,xc_yolo,yc_yolo,width_yolo,height_yolo
0,0,ID_007FAIEI,fruit_woodiness,87.0,87.5,228.0,311.0,0.392578,0.474609,0.445312,0.607422
1,1,ID_00G8K1V3,fruit_brownspot,97.5,17.5,245.0,354.5,0.429688,0.380371,0.478516,0.692383
2,2,ID_00WROUT9,fruit_brownspot,156.5,209.5,248.0,302.5,0.547852,0.704590,0.484375,0.590820
3,3,ID_00ZJEEK3,fruit_healthy,125.0,193.0,254.5,217.0,0.492676,0.588867,0.497070,0.423828
4,4,ID_018UIENR,fruit_brownspot,79.5,232.5,233.5,182.0,0.383301,0.631836,0.456055,0.355469


In [238]:
df[df["Image_ID"]=="ID_UVDMSYKR"]

,Unnamed: 0,Image_ID,class,xmin,ymin,width,height,xc_yolo,yc_yolo,width_yolo,height_yolo
3359,3359,ID_UVDMSYKR,fruit_woodiness,190.0,381.5,111.0,130.5,0.479492,0.872559,0.216797,0.254883
3360,3360,ID_UVDMSYKR,fruit_woodiness,198.0,168.0,167.5,225.0,0.550293,0.547852,0.327148,0.439453


In [24]:
df.groupby('Image_ID').filter(lambda x:x['Image_ID'].count()>1)

,Unnamed: 0,Image_ID,class,xmin,ymin,width,height,xc_yolo,yc_yolo,width_yolo,height_yolo
6,6,ID_02PX7GI8,fruit_brownspot,1.5,94.5,256.0,236.0,0.252930,0.415039,0.500000,0.460938
7,7,ID_02PX7GI8,fruit_brownspot,124.0,352.5,160.5,159.5,0.398926,0.844238,0.313477,0.311523
12,12,ID_05A7I1ZP,fruit_woodiness,321.0,275.5,145.5,194.5,0.769043,0.728027,0.284180,0.379883
13,13,ID_05A7I1ZP,fruit_woodiness,238.0,248.0,119.0,141.0,0.581055,0.622070,0.232422,0.275391
16,16,ID_05V8XX4L,fruit_woodiness,286.5,150.5,162.0,193.5,0.717773,0.482910,0.316406,0.377930
...,...,...,...,...,...,...,...,...,...,...,...
3898,3898,ID_ZYQFRA85,fruit_healthy,142.0,175.0,292.5,237.5,0.562988,0.573730,0.571289,0.463867
3901,3901,ID_ZZAB1FH1,fruit_healthy,96.0,175.5,289.5,222.0,0.470215,0.559570,0.565430,0.433594
3902,3902,ID_ZZAB1FH1,fruit_healthy,330.0,241.0,182.0,180.5,0.822266,0.646973,0.355469,0.352539
3903,3903,ID_ZZJZ2CV6,fruit_healthy,358.0,234.0,134.5,107.0,0.830566,0.561523,0.262695,0.208984


In [85]:
df[["Image_ID",'class']]

,Image_ID,class
0,ID_007FAIEI,fruit_woodiness
1,ID_00G8K1V3,fruit_brownspot
2,ID_00WROUT9,fruit_brownspot
3,ID_00ZJEEK3,fruit_healthy
4,ID_018UIENR,fruit_brownspot
...,...,...
3901,ID_ZZAB1FH1,fruit_healthy
3902,ID_ZZAB1FH1,fruit_healthy
3903,ID_ZZJZ2CV6,fruit_healthy
3904,ID_ZZJZ2CV6,fruit_healthy


In [100]:
def class_x(row):
    if row['class']=='fruit_healthy':
        return 1,0,0
    if row['class']=='fruit_brownspot':
        return 0,1,0
    if row['class']=='fruit_woodiness':
        return 0,0,1
    

# df_app=df_g.apply(lambda row:class_x(row),axis=1, result_type='expand')
# df_app.columns=['a','b','c']
# df_app

In [152]:
dff=df.apply(lambda x:class_x(x),axis=1,result_type="expand")
dff.columns=['a','b','c']


dfconcat=pd.concat([df,dff],axis=1)
dfconcat

,Unnamed: 0,Image_ID,class,xmin,ymin,width,height,xc_yolo,yc_yolo,width_yolo,height_yolo,a,b,c
0,0,ID_007FAIEI,fruit_woodiness,87.0,87.5,228.0,311.0,0.392578,0.474609,0.445312,0.607422,0,0,1
1,1,ID_00G8K1V3,fruit_brownspot,97.5,17.5,245.0,354.5,0.429688,0.380371,0.478516,0.692383,0,1,0
2,2,ID_00WROUT9,fruit_brownspot,156.5,209.5,248.0,302.5,0.547852,0.704590,0.484375,0.590820,0,1,0
3,3,ID_00ZJEEK3,fruit_healthy,125.0,193.0,254.5,217.0,0.492676,0.588867,0.497070,0.423828,1,0,0
4,4,ID_018UIENR,fruit_brownspot,79.5,232.5,233.5,182.0,0.383301,0.631836,0.456055,0.355469,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3901,3901,ID_ZZAB1FH1,fruit_healthy,96.0,175.5,289.5,222.0,0.470215,0.559570,0.565430,0.433594,1,0,0
3902,3902,ID_ZZAB1FH1,fruit_healthy,330.0,241.0,182.0,180.5,0.822266,0.646973,0.355469,0.352539,1,0,0
3903,3903,ID_ZZJZ2CV6,fruit_healthy,358.0,234.0,134.5,107.0,0.830566,0.561523,0.262695,0.208984,1,0,0
3904,3904,ID_ZZJZ2CV6,fruit_healthy,98.5,135.0,275.5,213.5,0.461426,0.472168,0.538086,0.416992,1,0,0


In [163]:
dfconcat['d']=np.full(len(dfconcat),1)
dfconcat

,Unnamed: 0,Image_ID,class,xmin,ymin,width,height,xc_yolo,yc_yolo,width_yolo,height_yolo,a,b,c,d
0,0,ID_007FAIEI,fruit_woodiness,87.0,87.5,228.0,311.0,0.392578,0.474609,0.445312,0.607422,0,0,1,1
1,1,ID_00G8K1V3,fruit_brownspot,97.5,17.5,245.0,354.5,0.429688,0.380371,0.478516,0.692383,0,1,0,1
2,2,ID_00WROUT9,fruit_brownspot,156.5,209.5,248.0,302.5,0.547852,0.704590,0.484375,0.590820,0,1,0,1
3,3,ID_00ZJEEK3,fruit_healthy,125.0,193.0,254.5,217.0,0.492676,0.588867,0.497070,0.423828,1,0,0,1
4,4,ID_018UIENR,fruit_brownspot,79.5,232.5,233.5,182.0,0.383301,0.631836,0.456055,0.355469,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3901,3901,ID_ZZAB1FH1,fruit_healthy,96.0,175.5,289.5,222.0,0.470215,0.559570,0.565430,0.433594,1,0,0,1
3902,3902,ID_ZZAB1FH1,fruit_healthy,330.0,241.0,182.0,180.5,0.822266,0.646973,0.355469,0.352539,1,0,0,1
3903,3903,ID_ZZJZ2CV6,fruit_healthy,358.0,234.0,134.5,107.0,0.830566,0.561523,0.262695,0.208984,1,0,0,1
3904,3904,ID_ZZJZ2CV6,fruit_healthy,98.5,135.0,275.5,213.5,0.461426,0.472168,0.538086,0.416992,1,0,0,1


In [235]:
#https://stackoverflow.com/questions/48819644/pandas-comparing-rows-within-groups

def funct(grp):
    i=grp.query("a==1")
    j=grp.query("b==1")
    
#     j=grp.query("(d==1)")
#     return i.join(j.set_index("Image_ID"),lsuffix="I",rsuffix="J",on="Image_ID",how="inner")
    return i
#     return i.join(j.set_index("Image_ID"),on="Image_ID",how="inner")
            
dffilt=dfconcat.groupby("Image_ID").apply(funct)

In [236]:
dffilt

Unnamed: 0     Image_ID          class   xmin   ymin  width  \
Image_ID                                                                        
ID_00ZJEEK3 3              3  ID_00ZJEEK3  fruit_healthy  125.0  193.0  254.5   
ID_05R05ZLJ 15            15  ID_05R05ZLJ  fruit_healthy   85.0   99.5  276.5   
ID_06EXUMWA 18            18  ID_06EXUMWA  fruit_healthy   43.5  122.0  287.0   
ID_06GLI7SR 19            19  ID_06GLI7SR  fruit_healthy  125.0  163.5  280.0   
ID_06N0H7F1 21            21  ID_06N0H7F1  fruit_healthy   91.0  168.5  309.5   
...                      ...          ...            ...    ...    ...    ...   
ID_ZZ5YBO5A 3900        3900  ID_ZZ5YBO5A  fruit_healthy  112.0  207.0  208.0   
ID_ZZAB1FH1 3901        3901  ID_ZZAB1FH1  fruit_healthy   96.0  175.5  289.5   
            3902        3902  ID_ZZAB1FH1  fruit_healthy  330.0  241.0  182.0   
ID_ZZJZ2CV6 3903        3903  ID_ZZJZ2CV6  fruit_healthy  358.0  234.0  134.5   
            3904        3904  ID_ZZJZ2CV6  fruit_healthy   98.5  135.0  275.5   

                  height   xc_yolo   yc_yolo  width_yolo  height_yolo  a  b  \
Image_ID                                                                      
ID_00ZJEEK3 3      217.0  0.492676  0.588867    0.497070     0.423828  1  0   
ID_05R05ZLJ 15     236.5  0.436035  0.425293    0.540039     0.461914  1  0   
ID_06EXUMWA 18     217.5  0.365234  0.450684    0.560547     0.424805  1  0   
ID_06GLI7SR 19     222.0  0.517578  0.536133    0.546875     0.433594  1  0   
ID_06N0H7F1 21     239.0  0.479980  0.562500    0.604492     0.466797  1  0   
...                  ...       ...       ...         ...          ... .. ..   
ID_ZZ5YBO5A 3900   176.5  0.421875  0.576660    0.406250     0.344727  1  0   
ID_ZZAB1FH1 3901   222.0  0.470215  0.559570    0.565430     0.433594  1  0   
            3902   180.5  0.822266  0.646973    0.355469     0.352539  1  0   
ID_ZZJZ2CV6 3903   107.0  0.830566  0.561523    0.262695     0.208984  1  0   
            3904   213.5  0.461426  0.472168    0.538086     0.416992  1  0   

                  c  d  
Image_ID                
ID_00ZJEEK3 3     0  1  
ID_05R05ZLJ 15    0  1  
ID_06EXUMWA 18    0  1  
ID_06GLI7SR 19    0  1  
ID_06N0H7F1 21    0  1  
...              .. ..  
ID_ZZ5YBO5A 3900  0  1  
ID_ZZAB1FH1 3901  0  1  
            3902  0  1  
ID_ZZJZ2CV6 3903  0  1  
            3904  0  1  

[1638 rows x 15 columns]

In [237]:
len(dffilt['aJ'].dropna()),len(dffilt['bJ'].dropna()),len(dffilt['cJ'].dropna()),len(dffilt['xminI'].dropna())

KeyError: 'aJ'

In [155]:
index=pd.MultiIndex.from_tuples(pd.MultiIndex.from_frame(dfconcat[["Image_ID",'class','a','b','c','xc_yolo','yc_yolo']]))
index
dfg=dfconcat[['class','Image_ID','a','b','c','xc_yolo','yc_yolo']].set_index(index)
# dfgex=dfg.apply(lambda x:class_x(x),axis=1,result_type="expand")
# dfgex.columns=['a','b','c']
dfg

class  \
ID_007FAIEI fruit_woodiness 0 0 1 0.392578 0.474609  fruit_woodiness   
ID_00G8K1V3 fruit_brownspot 0 1 0 0.429688 0.380371  fruit_brownspot   
ID_00WROUT9 fruit_brownspot 0 1 0 0.547852 0.704590  fruit_brownspot   
ID_00ZJEEK3 fruit_healthy   1 0 0 0.492676 0.588867    fruit_healthy   
ID_018UIENR fruit_brownspot 0 1 0 0.383301 0.631836  fruit_brownspot   
...                                                              ...   
ID_ZZAB1FH1 fruit_healthy   1 0 0 0.470215 0.559570    fruit_healthy   
                                  0.822266 0.646973    fruit_healthy   
ID_ZZJZ2CV6 fruit_healthy   1 0 0 0.830566 0.561523    fruit_healthy   
                                  0.461426 0.472168    fruit_healthy   
ID_ZZO9KIH9 fruit_woodiness 0 0 1 0.434082 0.576660  fruit_woodiness   

                                                        Image_ID  a  b  c  \
ID_007FAIEI fruit_woodiness 0 0 1 0.392578 0.474609  ID_007FAIEI  0  0  1   
ID_00G8K1V3 fruit_brownspot 0 1 0 0.429688 0.380371  ID_00G8K1V3  0  1  0   
ID_00WROUT9 fruit_brownspot 0 1 0 0.547852 0.704590  ID_00WROUT9  0  1  0   
ID_00ZJEEK3 fruit_healthy   1 0 0 0.492676 0.588867  ID_00ZJEEK3  1  0  0   
ID_018UIENR fruit_brownspot 0 1 0 0.383301 0.631836  ID_018UIENR  0  1  0   
...                                                          ... .. .. ..   
ID_ZZAB1FH1 fruit_healthy   1 0 0 0.470215 0.559570  ID_ZZAB1FH1  1  0  0   
                                  0.822266 0.646973  ID_ZZAB1FH1  1  0  0   
ID_ZZJZ2CV6 fruit_healthy   1 0 0 0.830566 0.561523  ID_ZZJZ2CV6  1  0  0   
                                  0.461426 0.472168  ID_ZZJZ2CV6  1  0  0   
ID_ZZO9KIH9 fruit_woodiness 0 0 1 0.434082 0.576660  ID_ZZO9KIH9  0  0  1   

                                                      xc_yolo   yc_yolo  
ID_007FAIEI fruit_woodiness 0 0 1 0.392578 0.474609  0.392578  0.474609  
ID_00G8K1V3 fruit_brownspot 0 1 0 0.429688 0.380371  0.429688  0.380371  
ID_00WROUT9 fruit_brownspot 0 1 0 0.547852 0.704590  0.547852  0.704590  
ID_00ZJEEK3 fruit_healthy   1 0 0 0.492676 0.588867  0.492676  0.588867  
ID_018UIENR fruit_brownspot 0 1 0 0.383301 0.631836  0.383301  0.631836  
...                                                       ...       ...  
ID_ZZAB1FH1 fruit_healthy   1 0 0 0.470215 0.559570  0.470215  0.559570  
                                  0.822266 0.646973  0.822266  0.646973  
ID_ZZJZ2CV6 fruit_healthy   1 0 0 0.830566 0.561523  0.830566  0.561523  
                                  0.461426 0.472168  0.461426  0.472168  
ID_ZZO9KIH9 fruit_woodiness 0 0 1 0.434082 0.576660  0.434082  0.576660  

[3906 rows x 7 columns]

In [143]:
dfgex

a  b  c
ID_007FAIEI fruit_woodiness  0  0  1
ID_00G8K1V3 fruit_brownspot  0  1  0
ID_00WROUT9 fruit_brownspot  0  1  0
ID_00ZJEEK3 fruit_healthy    1  0  0
ID_018UIENR fruit_brownspot  0  1  0
...                         .. .. ..
ID_ZZAB1FH1 fruit_healthy    1  0  0
            fruit_healthy    1  0  0
ID_ZZJZ2CV6 fruit_healthy    1  0  0
            fruit_healthy    1  0  0
ID_ZZO9KIH9 fruit_woodiness  0  0  1

[3906 rows x 3 columns]

In [112]:
df[df["Image_ID"]=="ID_ZZAB1FH1"]

,Unnamed: 0,Image_ID,class,xmin,ymin,width,height,xc_yolo,yc_yolo,width_yolo,height_yolo
3901,3901,ID_ZZAB1FH1,fruit_healthy,96.0,175.5,289.5,222.0,0.470215,0.559570,0.565430,0.433594
3902,3902,ID_ZZAB1FH1,fruit_healthy,330.0,241.0,182.0,180.5,0.822266,0.646973,0.355469,0.352539


In [43]:
f="ID_00ZJEEK3.jpg"
print(f[0:-4])
s=df[df['Image_ID']==f[0:-4]]
s

ID_00ZJEEK3


,Unnamed: 0,Image_ID,class,xmin,ymin,width,height,xc_yolo,yc_yolo,width_yolo,height_yolo
3,3,ID_00ZJEEK3,fruit_healthy,125.0,193.0,254.5,217.0,0.492676,0.588867,0.49707,0.423828


In [44]:
s=df.loc[df["Image_ID"]=="ID_00ZJEEK3"]
s

,Unnamed: 0,Image_ID,class,xmin,ymin,width,height,xc_yolo,yc_yolo,width_yolo,height_yolo
3,3,ID_00ZJEEK3,fruit_healthy,125.0,193.0,254.5,217.0,0.492676,0.588867,0.49707,0.423828


In [45]:
s.iloc[0],type(s.iloc[0])

(Unnamed: 0                 3
 Image_ID         ID_00ZJEEK3
 class          fruit_healthy
 xmin                     125
 ymin                     193
 width                  254.5
 height                   217
 xc_yolo             0.492676
 yc_yolo             0.588867
 width_yolo           0.49707
 height_yolo         0.423828
 Name: 3, dtype: object,
 pandas.core.series.Series)

In [50]:
ss=s.iloc[0]
ss

Unnamed: 0                 3
Image_ID         ID_00ZJEEK3
class          fruit_healthy
xmin                     125
ymin                     193
width                  254.5
height                   217
xc_yolo             0.492676
yc_yolo             0.588867
width_yolo           0.49707
height_yolo         0.423828
Name: 3, dtype: object

In [49]:
gshape=(7,7)
gw=1/7
gh=1/7
gw,gh

(0.14285714285714285, 0.14285714285714285)

In [60]:
tgx=ss['xc_yolo']/gw
tgy=ss['yc_yolo']/gh
tgx,tgy

(3.44873046875, 4.1220703125)

In [59]:
gx=ss['xc_yolo']//gw
gy=ss['yc_yolo']//gh
gx,gy

(3.0, 4.0)

In [58]:
xg=ss['xc_yolo']%gw
yg=ss['yc_yolo']%gh
xg,yg

(0.06410435267857145, 0.017438616071428603)

In [65]:
gx*gw+xg,ss['xc_yolo']

(0.49267578125, 0.49267578125)

In [68]:
wc=ss['width_yolo']
wc/gw,wc

(3.4794921875, 0.4970703125)

In [71]:
ss['width']/512,wc/gw/7

(0.4970703125, 0.4970703125)

In [6]:
#place a (7x7) grid on the image
#convert xc and yc to the grid coords
r=df.iloc[2]
xc=r['xc_yolo']
yc=r['yc_yolo']
wd=r['width_yolo']
ht=r['height_yolo']
gshape=(7,7)
iwidth=512
iheight=512

xc,yc,wd,ht

(0.5478515625, 0.70458984375, 0.484375, 0.5908203125)

In [7]:
gwidth=1/gshape[0]           #taken 1 here since the image is normalized in yolo coords. i.e all coords between (0,1)
gheight=1/gshape[1]
gwidth,gheight

(0.14285714285714285, 0.14285714285714285)

In [8]:
xc//gwidth   #<== grid-x where the point occurs. i.e a point that is at ( 0.54785,0.704 ) in the image falls inside the the grid at (3,4) when a 7x7 grid is placed on it

3.0

In [9]:
yc//gheight  #<== grid y where the point occurs. i.e a point that is at ( 0.54785,0.704 ) in the image falls inside the the grid at (3,4) when a 7x7 grid is placed on it

4.0

In [10]:
xc%gwidth,xc//gwidth

(0.11928013392857145, 3.0)

In [11]:
xc//gwidth*gwidth+xc%gwidth

0.5478515625

In [12]:
x=xc%gwidth #<== value of xc in inside the grid that needs to be stored in layer-1
y=yc%gheight #<== value of yc inside the grid that contains (xc,yc) .This is the value that needs to be stored in layer-2
w=wd/gwidth  #<==the length gets scaled in the grid coordinate system
h=ht/gheight  #<==length scaling
x,y,w,h

(0.11928013392857145, 0.1331612723214286, 3.390625, 4.1357421875)

In [14]:
gd=df.groupby(["Image_ID"])
df_g=gd.filter(lambda x: (x["Image_ID"].count()>2))
df_g
# .apply(lambda row:class_x(row),axis=1, result_type='expand')

,Unnamed: 0,Image_ID,class,xmin,ymin,width,height,xc_yolo,yc_yolo,width_yolo,height_yolo
23,23,ID_07HGJOFB,fruit_healthy,283.0,389.5,220.5,122.5,0.768066,0.880371,0.430664,0.239258
24,24,ID_07HGJOFB,fruit_healthy,275.5,10.0,198.0,183.0,0.731445,0.198242,0.386719,0.357422
25,25,ID_07HGJOFB,fruit_healthy,115.0,206.0,204.0,168.0,0.423828,0.566406,0.398438,0.328125
56,56,ID_0JOUSB83,fruit_healthy,80.5,135.5,318.5,255.5,0.468262,0.514160,0.622070,0.499023
57,57,ID_0JOUSB83,fruit_healthy,210.0,0.0,192.0,79.0,0.597656,0.077148,0.375000,0.154297
...,...,...,...,...,...,...,...,...,...,...,...
3885,3885,ID_ZTQT7QME,fruit_healthy,106.5,171.0,256.0,221.0,0.458008,0.549805,0.500000,0.431641
3886,3886,ID_ZTQT7QME,fruit_healthy,26.5,15.5,185.5,153.0,0.232910,0.179688,0.362305,0.298828
3887,3887,ID_ZUT4UN5I,fruit_woodiness,147.5,126.0,158.0,235.5,0.442383,0.476074,0.308594,0.459961
3888,3888,ID_ZUT4UN5I,fruit_woodiness,0.0,346.0,240.0,166.0,0.234375,0.837891,0.468750,0.324219


In [16]:
dg=pd.concat([df,df_app],axis=1)# df_g
dg

,Unnamed: 0,Image_ID,class,xmin,ymin,width,height,xc_yolo,yc_yolo,width_yolo,height_yolo,a,b,c
0,0,ID_007FAIEI,fruit_woodiness,87.0,87.5,228.0,311.0,0.392578,0.474609,0.445312,0.607422,NaN,NaN,NaN
1,1,ID_00G8K1V3,fruit_brownspot,97.5,17.5,245.0,354.5,0.429688,0.380371,0.478516,0.692383,NaN,NaN,NaN
2,2,ID_00WROUT9,fruit_brownspot,156.5,209.5,248.0,302.5,0.547852,0.704590,0.484375,0.590820,NaN,NaN,NaN
3,3,ID_00ZJEEK3,fruit_healthy,125.0,193.0,254.5,217.0,0.492676,0.588867,0.497070,0.423828,NaN,NaN,NaN
4,4,ID_018UIENR,fruit_brownspot,79.5,232.5,233.5,182.0,0.383301,0.631836,0.456055,0.355469,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3901,3901,ID_ZZAB1FH1,fruit_healthy,96.0,175.5,289.5,222.0,0.470215,0.559570,0.565430,0.433594,NaN,NaN,NaN
3902,3902,ID_ZZAB1FH1,fruit_healthy,330.0,241.0,182.0,180.5,0.822266,0.646973,0.355469,0.352539,NaN,NaN,NaN
3903,3903,ID_ZZJZ2CV6,fruit_healthy,358.0,234.0,134.5,107.0,0.830566,0.561523,0.262695,0.208984,NaN,NaN,NaN
3904,3904,ID_ZZJZ2CV6,fruit_healthy,98.5,135.0,275.5,213.5,0.461426,0.472168,0.538086,0.416992,NaN,NaN,NaN


In [53]:
gd[gd['class']==True].iloc[0]

Unnamed: 0     True
class          True
xmin           True
ymin           True
width          True
height         True
xc_yolo        True
yc_yolo        True
width_yolo     True
height_yolo    True
Name: ID_02PX7GI8, dtype: bool